<a href="https://colab.research.google.com/github/julotec/hmk10/blob/main/hmk2_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# Wczytanie danych
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Przygotowanie danych dla VGG16 (przekształcenie do formatu 3-kanałowego)
train_images = np.stack([train_images]*3, axis=-1)
test_images = np.stack([test_images]*3, axis=-1)

# Konwersja etykiet na kategorie
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

4422102/4422102 [==============================] - 0s 0us/step


In [4]:
# Definiowanie funkcji do zmiany rozdzielczości i normalizacji
def preprocess_function(image, label):
    image = tf.image.resize(image, (128, 128))
    image = image / 255.0
    return image, label

# Tworzenie datasetów
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(preprocess_function).batch(16).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
val_dataset = val_dataset.map(preprocess_function).batch(16).prefetch(tf.data.AUTOTUNE)

In [5]:
# Użycie pretrenowanego modelu VGG16
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
vgg_base.trainable = False  # Zamrożenie wag modelu

# Dodanie własnych warstw gęstych na szczycie bazy konwolucyjnej
model = models.Sequential([
    vgg_base,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model.summary()

58889256/58889256 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 16814666 (64.14 MB)
Trainable params: 2099978 (8.01 MB)
Non-trainable params: 14714688 (56.13 MB)
___________

In [6]:
# Kompilacja modelu
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Definiowanie kroków na epokę
steps_per_epoch = len(train_images) // 16  # Ustawienie kroków na podstawie batch size

# Trenowanie modelu z ograniczoną ilością kroków na epokę
history = model.fit(train_dataset, epochs=10, steps_per_epoch=steps_per_epoch, validation_data=val_dataset)

# Ewaluacja modelu
test_loss, test_acc = model.evaluate(val_dataset)
print(f"VGG16-based CNN Test accuracy: {test_acc}")

Epoch 1/10
3750/3750 [==============================] - 120s 30ms/step - loss: 0.5120 - accuracy: 0.8169 - val_loss: 0.3656 - val_accuracy: 0.8690
Epoch 2/10
3750/3750 [==============================] - 106s 28ms/step - loss: 0.4015 - accuracy: 0.8555 - val_loss: 0.3153 - val_accuracy: 0.8858
Epoch 3/10
3750/3750 [==============================] - 106s 28ms/step - loss: 0.3670 - accuracy: 0.8663 - val_loss: 0.3017 - val_accuracy: 0.8900
Epoch 4/10
3750/3750 [==============================] - 106s 28ms/step - loss: 0.3465 - accuracy: 0.8738 - val_loss: 0.2887 - val_accuracy: 0.8943
Epoch 5/10
3750/3750 [==============================] - 112s 30ms/step - loss: 0.3282 - accuracy: 0.8816 - val_loss: 0.2935 - val_accuracy: 0.8973
Epoch 6/10
3750/3750 [==============================] - 106s 28ms/step - loss: 0.3159 - accuracy: 0.8837 - val_loss: 0.2991 - val_accuracy: 0.8922
Epoch 7/10
3750/3750 [==============================] - 112s 30ms/step - loss: 0.3089 - accuracy: 0.8864 - val_loss: 0